In [3]:
import os

In [4]:
os.chdir('../')

In [6]:
%pwd

'/Users/mohamedabrar/Desktop/ml_project/Chicken_Disease-Classification/Chicken-Disease-Classification'

In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [14]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [15]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [16]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [28]:
import certifi

os.environ['SSL_CERT_FILE'] = certifi.where()

import ssl
import urllib.request

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        self.ssl_context = ssl.create_default_context(cafile=certifi.where())

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            try:
                with urllib.request.urlopen(self.config.source_URL, context=self.ssl_context) as response:
                    with open(self.config.local_data_file, 'wb') as out_file:
                        data = response.read()
                        out_file.write(data)
                logger.info(f"{self.config.local_data_file} downloaded!")
            except Exception as e:
                logger.error(f"Failed to download file: {e}")
        else:
            file_size = os.path.getsize(self.config.local_data_file)
            logger.info(f"File already exists of size: {file_size} bytes")
    
    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        try:
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            logger.info(f"Extracted files to {unzip_path}")
        except zipfile.BadZipFile as e:
            logger.error(f"Failed to extract zip file: {e}")

In [29]:
pip install certifi

Note: you may need to restart the kernel to use updated packages.


In [30]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-06-11 00:57:52,641: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-06-11 00:57:52,643: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-11 00:57:52,644: INFO: common: created directory at: artifacts]
[2024-06-11 00:57:52,645: INFO: common: created directory at: artifacts/data_ingestion]
[2024-06-11 00:57:57,730: INFO: 653769923: artifacts/data_ingestion/data.zip downloaded!]
[2024-06-11 00:57:57,836: INFO: 653769923: Extracted files to artifacts/data_ingestion]
